In [22]:
import agentpy as ap
import matplotlib.pyplot as plt
import numpy as np

# Visualization
import seaborn as sns
import IPython

In [23]:
#Funciones Auxiliares
def iteratorlist(node):

  out = []

  for i in(node):
    out.append(i)

  return out

In [24]:
class Node(ap.Agent):

    #Constructor
    def setup(self):

        #Reglas de desplazamiento
        self.desp = [[0, 1], [0, -1], [1, 0], [-1, 0]]

        #Reglas de desplazamiento
        self.xlim = 20
        self.ylim = 20

        #Crear ID
        self.id = None

        #Ruta de camino
        self.parents = []

        #Tipo de rama
        self.branch = 0

    #Asignar ID
    def assign_id(self, id):

        #Asignar ID
        self.id = int(id)
        self.parents.append(int(id))

    #Reglas de desplazamiento
    def setup_rules(self, xlim, ylim):

        #Reglas de desplazamiento
        self.xlim = xlim
        self.ylim = ylim

    #Obtener posicion
    def get_position(self, world):

        #Get position
        position = world.positions[self]

        return position

    #Obtener estado siguiente
    def get_next_state(self, world):

        #Obtener posicion
        position = self.get_position(world)

        #Obtener valores de alrededores
        values = []
        for i in range(len(self.desp)):

          #Obtener posicion tras desplazamiento
          pos_x = position[0] + self.desp[i][0]
          pos_y = position[1] + self.desp[i][1]

          #Obtener condicion
          if((pos_x < 0) or (pos_y < 0) or (pos_x > self.xlim - 1) or (pos_y > self.ylim - 1)):
            values.append(-1)
          else:
            #Obtener posicion siguiente
            value = world.agents[(pos_x, pos_y)].condition
            for i in value:
              values.append(i)
              break

        return values

    #Propagar trayectoria
    def propagate(self, world):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Propagar camino
      for i in range(len(surroundings)):

        #Posicion del vecindario
        pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])
        
        # 0: Libre. 1
        # 1: Obstaculo.
        # 2: Activo. 6
        # 3: Camino. 7
        # 4: Objetivo. 4
        # 5: Solucion. 5
        # 6: 9

        #Espacio libre
        if(surroundings[i] == 0):

          #Cambiar condicion a nodo activo (2)
          world.agents[pos].condition = 2

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents

        #Nodo objetivo
        if(surroundings[i] == 4):

          #Cambiar condicion de nodo objetivo
          world.agents[pos].condition = 5

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents
          break

    #Propagar trayectoria
    def propagate_bidirectional(self, world):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Propagar camino
      for i in range(len(surroundings)):

        #Posicion del vecindario
        pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])

        #Espacio libre
        if(surroundings[i] == 0):

          #Cambiar condicion a nodo activo (2)
          world.agents[pos].condition = 2
          world.agents[pos].branch = self.branch

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents

        #Nodo activo
        if(surroundings[i] == 2):

          #Obtener condicion del vecindario
          current_branch = iteratorlist(world.agents[pos].branch)

          #Verificar si la rama pertenece a otro origen
          if(current_branch[0] != self.branch):
            #Cambiar condicion de nodo objetivo
            world.agents[pos].condition = 5
            world.agents[pos].parents += self.parents
            break

    #Propagar hacia el mejor
    def propagate_best(self, world, goal, bidirectional = False):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener posicion del objetivo
      goal_ob = goal.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Buscar nodo mas cercano
      best_pos = None
      min_cost = 1000
      for i in range(len(surroundings)):

        #Obtener posicion
        pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])

        #Camino libre
        if(surroundings[i] == 0):

          #Calcular costo al objetivo
          cost = 0.5*np.sum(np.power(np.array(pos) - np.array(goal_ob), 2))

          #Guardar mejor opcion
          if(cost < min_cost):
            best_pos = pos
            min_cost = cost

        #Nodo objetivo
        if(not(bidirectional)):
          if(surroundings[i] == 4):

            #Cambiar condicion de nodo objetivo
            world.agents[pos].condition = 5

            #Heredar camino de regreso
            world.agents[pos].parents += self.parents
            break
        else:
          #Nodo activo
          if(surroundings[i] == 2):

            #Obtener condicion del vecindario
            current_branch = iteratorlist(world.agents[pos].branch)

            #Verificar si la rama pertenece a otro origen
            if(current_branch[0] != self.branch):
              #Cambiar condicion de nodo objetivo
              world.agents[pos].condition = 5
              world.agents[pos].parents += self.parents
              break

      #Crecer nodo
      if(best_pos != None):

        #Cambiar condicion a nodo activo (2)
        world.agents[best_pos].condition = 2
        world.agents[best_pos].branch = self.branch

        #Heredar camino de regreso
        world.agents[best_pos].parents += self.parents

      #Matar nodo si no tiene ningun camino
      if(best_pos == None):
        self.condition = 6

    #Propagacion random
    def propagate_random(self, world):

      #Obtener posicion del nodo
      position = self.get_position(world)

      #Obtener alrededores
      surroundings = self.get_next_state(world)

      #Obtener espacios libres
      free_spaces = []
      for i in range(len(surroundings)):

        #Camino libre
        if(surroundings[i] == 0):
          free_spaces.append(i)

        #Nodo objetivo
        if(surroundings[i] == 4):

          #Obtener posicion
          pos = (position[0] + self.desp[i][0],
              position[1] + self.desp[i][1])

          #Cambiar condicion de nodo objetivo
          world.agents[pos].condition = 5

          #Heredar camino de regreso
          world.agents[pos].parents += self.parents
          break

      #Elegir espacio al azar
      if(len(free_spaces) > 0):

        #Elegir direccion
        index = free_spaces[np.random.randint(len(free_spaces))]

        #Obtener posicion
        pos = (position[0] + self.desp[index][0],
              position[1] + self.desp[index][1])

        #Cambiar condicion a nodo activo (2)
        world.agents[pos].condition = 2
        world.agents[pos].branch = self.branch

        #Heredar camino de regreso
        world.agents[pos].parents += self.parents

      #Matar nodo si no tiene ningun camino
      else:
        self.condition = 6

In [25]:
class RandomMap(ap.Model):

    #Metodo de configuracion
    def setup(self):

      #Obtener tipo de busqueda
      self.mode = self.p['mode']
      self.algorithm = self.p['algorithm']
      self.gains = self.p['gains']

      #Crear agentes del piso
      n_agents = int(self.p['dim']**2)
      self.floor_agents = ap.AgentList(self, n_agents, Node)

      #Crear espacio
      self.floor = ap.Grid(self, [self.p['dim']]*2, track_empty=True)
      self.floor.add_agents(self.floor_agents)

      #Definir espacio libre
      self.floor_agents.condition = 0

      #Definir obstaculos
      obstacle_index = np.random.randint(len(self.floor_agents), size = int(self.p['obs_den']*n_agents))
      for i in range(obstacle_index.shape[0]):
        self.floor_agents[obstacle_index[i]].condition = 1

      #Crear nodo raiz
      self.start_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.start_index].condition = 2

      #Crear nodo objetivo
      self.goal_index = np.random.randint(len(self.floor_agents))
      self.floor_agents[self.goal_index].condition = 4

      #Asignar ID unico a nodos
      for i in range(len(self.floor_agents)):
        self.floor_agents[i].assign_id(i)

      #Asignar ramas
      self.floor_agents[self.start_index].branch = 0
      self.floor_agents[self.goal_index].branch = 1

    #Iteracion
    def step(self):

      #Buscar nodos activos
      active_nodes = []

      #1- Busqueda por anchura
      if(self.algorithm == 0):
        for agent in self.floor_agents:

          #Verificar condicion
          if(agent.condition == 2):
            active_nodes.append(agent)
          if(self.mode == 1):
            if(agent.condition == 4):
                active_nodes.append(agent)

      #2- Busqueda por el mejor
      if(self.algorithm == 1 or self.algorithm == 3):

        #Inicializar salida
        min_error = 1000
        best_agent = None
        for agent in self.floor_agents:

          #Verificar condicion
          if(agent.condition == 2):

            #Obtener posicion de agente y objetivo
            pos = agent.get_position(self.floor)
            pos_goal = self.floor_agents[self.goal_index].get_position(self.floor)

            #Calcular error
            error = 0.5*np.sum(np.power(np.array(pos) - np.array(pos_goal), 2))

            #Guardar mejor
            if(error < min_error):
              best_agent = agent
              min_error = error

        #Marcar mejor agente
        active_nodes.append(best_agent)

      #3- Busqueda A*
      if(self.algorithm == 2):

        #Obtener posicion del origen
        origin_pos = self.floor_agents[self.start_index].get_position(self.floor)

        #Inicializar salida
        min_error = 1000
        best_agent = None
        for agent in self.floor_agents:

          #Verificar condicion
          if(agent.condition == 2):

            #Obtener posicion de agente y objetivo
            pos = agent.get_position(self.floor)
            pos_goal = self.floor_agents[self.goal_index].get_position(self.floor)

            #Calcular error
            error_agent = 0.5*np.sum(np.power(np.array(pos) - np.array(pos_goal), 2))
            error_orig = 0.5*np.sum(np.power(np.array(pos) - np.array(origin_pos), 2))
            error = self.gains[0]*error_agent + self.gains[1]*error_orig

            #Guardar mejor
            if(error < min_error):
              best_agent = agent
              min_error = error

        #Marcar mejor agente
        active_nodes.append(best_agent)

      #Ejecutar metodo de busqueda
      #a) Busqueda Unidireccional
      if(self.mode == 0):

        #Propagar nodos activos
        for agent in active_nodes:

          #1- Busqueda por anchura
          if(self.algorithm == 0):
            #Crecer ramas
            agent.propagate(self.floor)

          #2- Busqueda primero el mejor
          if(self.algorithm == 1):

            #Crecer ramas hacia el mejor
            agent.propagate_best(self.floor, self.floor_agents[self.goal_index])

          #3- Busqueda A*
          if(self.algorithm == 2):

            #Crecer ramas hacia el mejor
            agent.propagate_best(self.floor, self.floor_agents[self.goal_index])

          #4- Busqueda aleatoria
          if(self.algorithm == 3):

            #Crecer ramas aleatoriamente
            agent.propagate_random(self.floor)

      #b) Busqueda por anchura bidireccional
      if(self.mode == 1):

        #Propagar nodos activos
        for agent in active_nodes:

          #1- Busqueda por anchura
          if(self.algorithm == 0):
            #Crecer ramas
            agent.propagate_bidirectional(self.floor)

          #2- Busqueda primero el mejor
          if(self.algorithm == 1):
            #Crecer ramas
            agent.propagate_best(self.floor, self.floor_agents[self.goal_index], bidirectional = True)

          #3- Busqueda A*
          if(self.algorithm == 2):
            pass

          #4- Busqueda aleatoria
          if(self.algorithm == 3):
            pass

      #Buscar si algoritmo se completo
      for agent in self.floor_agents:

        #Verificar condicion 5
        if(agent.condition == 5):

          #Recuperar camino de vuelta
          self.draw_path(agent)

          #Detener ejecucion
          self.stop()

    #Marcar camino
    def draw_path(self, node):

      #Obtener camino del objetivo
      path = []
      for parent in node.parents:
        path.append(parent)

      #Cambiar estado de los nodos
      for j in range(len(path)):
        for agent in self.floor_agents:
          if(agent.id == path[j]):
            agent.condition = 3

    #Finalizar simulacion
    def end(self):

        print("Camino encontrado!")
        self.floor_agents[self.goal_index].condition == 3

In [26]:
# Inicializar parametros
parameters = {
    'obs_den': 0.3, # Percentage of grid covered by obstacles
    'dim': 20, # Height and length of the grid
    'mode': 1,  #Modo: 0-Unidireccional, 1-Bidireccional
    'algorithm': 2, #Algoritmo: 0-Anchura, 1-Mejor, 2-A*, 3-Random
    'gains': [0.9, 0.1]  #Goal gain, origin gain
}

In [27]:
#Crear instancia
model = RandomMap(parameters)

In [28]:
# Create single-run animation with custom colors
def rgb2hex(r, g, b):

  var = '#%02x%02x%02x' % (r, g, b)

  return var

# Create single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.floor.attr_grid('condition')
    color_dict = {0:rgb2hex(255, 255, 255), 1:rgb2hex(0, 0, 0), 2:rgb2hex(128, 128, 128), 3:rgb2hex(0, 255, 0), 4:rgb2hex(255, 0, 0), 5:rgb2hex(0, 0, 255), 6:rgb2hex(0, 255, 255), None:rgb2hex(255, 255, 255)}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

fig, ax = plt.subplots()
animation = ap.animate(model, fig, ax, animation_plot, steps = 100)
IPython.display.HTML(animation.to_jshtml(fps=15))